# 1.import

In [61]:
import pandas as pd

# 2.data load

In [62]:
test = pd.read_csv('data_files/test.csv')
gender = pd.read_csv('data_files/gender_submission.csv')
train = pd.read_csv('data_files/train.csv')
test_ids = test["PassengerId"]

# 3.execution

## initial view

trying to figure out who survived and who didn't survive

In [63]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [64]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [65]:
gender.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## cleaning data

In [66]:
def clean(data):
    data = data.drop(["Ticket","Cabin","Name","PassengerId"], axis = 1)
    cols = ["SibSp","Parch","Fare","Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace = True)
    data.Embarked.fillna("U", inplace = True)
    return data



In [67]:
train = clean(train)
test = clean(test)

In [68]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


## labeling

In [69]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


In [70]:
cols = ['Sex','Embarked']
for col in cols:
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])
    print(le.classes_)

train.head(5)

['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


## sklearn(sickit-learn)

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [72]:
y = train["Survived"]
X = train.drop("Survived", axis = 1)
X_train, X_val, y_train, y_val  = train_test_split(X, y, test_size= 0.2, random_state=42)


In [73]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)


In [74]:
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8100558659217877

## submissions

In [75]:
submission_preds = clf.predict(test)

In [76]:
df = pd.DataFrame({"passengerId": test_ids.values,
                   "Survived": submission_preds,
                   })

In [77]:
df.to_csv("submission.csv", index=False)